In [1]:
import pandas as pd
from IPython.display import display
import sqlite3
import pprint
import re
import json
import datetime

raw_data_path = r"C:\Users\natha\iCloudDrive\Health\nomie_export\nomie-5.6.4-2023-01-10-15_23.json"

In [2]:
with open(raw_data_path) as json_file:
    data = json.load(json_file)
    df = pd.DataFrame(data["events"])

print(df)

             _id                note            end          start  score  \
0     a42c0adde0  #sleep_quality(93)  1645971808544  1645971807544      1   
1     37bdf1c3fd      #caffeine(550)  1645971816012  1645971815012      0   
2     0136cb75ae             #pooped  1645981062084  1645981061084      0   
3     b1d040bc43        #anxiety_med  1645993719008  1645993718008      0   
4     016ccbf388        #anxiety_med  1645993720508  1645993719508      0   
...          ...                 ...            ...            ...    ...   
2137  9b6a8f8864             #pooped  1673319985380  1673319984380      0   
2138  1e67ef7454  #sleep_quality(67)  1673356142034  1673356141034      1   
2139  a19169e9a1             #pooped  1673356143419  1673356142419      0   
2140  47fb59f600             #pooped  1673367139613  1673367138613      0   
2141  fe0399a189      #caffeine(750)  1673367144554  1673367143554      0   

       lat   lng location  offset modified source  
0     None  None       

In [3]:
# # Format and make one of each date
# dates = set()
# for datetimestamp in df["start"]:
#     date = datetime.date.fromtimestamp(datetimestamp / 1e3).strftime('%Y-%m-%d')
#     if date:
#         dates.add(date)
# #         print(date.group())

# sorted_dates = sorted(dates)
# pprint.pprint(sorted_dates)

In [4]:
# Setup an empty master dataframe
mod_data = pd.DataFrame()
mod_data = df

mod_data["date"] = pd.to_datetime(df["start"], unit="ms").dt.strftime("%Y-%m-%d")
mod_data["val"] = df["note"].str.extract("(\d+)")
mod_data["note"] = df["note"].str.extract("([a-z]+)")

# print(pd.to_datetime(df["start"], unit="ms").dt.strftime("%Y-%m-%d"))


#mod_data["date"] = all_data["start"].to_datetime()
mod_data = df.drop(columns=['_id', 'start', 'end', 'score', 'lat', 'lng', 'location', 'offset', 'modified', 'source'])

display(mod_data)

,note,date,val
0,sleep,2022-02-27,93
1,caffeine,2022-02-27,550
2,pooped,2022-02-27,NaN
3,anxiety,2022-02-27,NaN
4,anxiety,2022-02-27,NaN
...,...,...,...
2137,pooped,2023-01-10,NaN
2138,sleep,2023-01-10,67
2139,pooped,2023-01-10,NaN
2140,pooped,2023-01-10,NaN


In [5]:
healthdb = sqlite3.connect('HealthDB.db')
cur = healthdb.cursor()

cur.execute("DROP TABLE IF EXISTS nomie5")

mod_data.to_sql(name='nomie5', con=healthdb)
print("done")

done
